In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from calendar import monthrange

In [2]:
calendar_df = pd.read_excel("calendar_setting.xlsx", sheet_name = '1')
current_wop_df = pd.read_excel("2021.02.11 Приоритет запусков-остановок.xlsm", sheet_name = 'Потенциал')

In [33]:
next_month = 8
days_in_next_month = monthrange(2021, next_month)[1]
rho_oil = 0.947
df_well_rates = pd.DataFrame()

In [34]:
# Well class

class Well():
    
    def __init__(self, well_name, current_PI, current_WCT, 
                 current_GOR, current_Pr, current_BHP):
        
        self.well_name = well_name
        self.Pr = current_Pr
        
        # Current WOP
        
        self.current_PI = current_PI
        self.current_WCT = current_WCT
        self.current_GOR = current_GOR
        self.current_BHP = current_BHP
        
        # Planned WOP
        
        self.pl_PI = None
        self.pl_WCT = None
        self.pl_GOR = None
        self.pl_BHP = None
        
        # Schedule
        
        self.schedule = []
        
        # Forecasted WOP
        
        self.l_rate = []
        self.o_rate = []
        self.g_rate = []
        
        # Cumulative rates 
        
        self.c_l_rate = 0 
        self.c_o_rate = 0
        self.c_g_rate = 0  
        
# Assign Planned WOP        
        
    def assign_planned_wop(self):
                
        for i in range(calendar_df.shape[0]):
    
            if self.well_name == calendar_df['Well'][i]:
    
                if not pd.isnull(calendar_df['Changed WOP'][i]):
            
                    # Select changed WOP
                    well_changed_wop_str = calendar_df['Changed WOP'][i]

                    # Split str to list
                    well_changed_wop = well_changed_wop_str.split(',')
                    
                    # Check for 'S'
                    if well_changed_wop[0] == 'S':

                        self.pl_PI = self.current_PI
                    else:
                        self.pl_PI = int(well_changed_wop[0])

                    self.pl_BHP = int(well_changed_wop[1])
                    self.pl_WCT = int(well_changed_wop[2])
                    self.pl_GOR = int(well_changed_wop[3])

# Assigning schedule
                    
    def assign_schedule(self):
        
        schedule_series = calendar_df.loc[calendar_df['Well'].isin([self.well_name])]
    
        for i in range(1,days_in_next_month+1):
            
            self.schedule.append(schedule_series.iloc[0][i])
            
# Assigning rates

    def assign_rates(self):
        
        for i in self.schedule:
    
            # If 1 - the same WOP as at the end of the month
            if i == 1:
                
                l_rate = self.current_PI * (self.Pr - self.current_BHP)
                o_rate = l_rate * (1 - self.current_WCT/100) * rho_oil
                g_rate = o_rate * self.current_GOR / 1000
                
                self.l_rate.append(l_rate)
                self.o_rate.append(o_rate)
                self.g_rate.append(g_rate)
                
            # If 2 - the changed WOP as at the end of the month    
            elif i == 2:
                
                l_rate = self.pl_PI * (self.Pr - self.pl_BHP)
                o_rate = l_rate * (1 - self.pl_WCT/100) * rho_oil
                g_rate = o_rate * self.pl_GOR / 1000
                
                self.l_rate.append(l_rate)
                self.o_rate.append(o_rate)
                self.g_rate.append(g_rate)
        
            # If 0 - well is stopped  
            elif i == 0:
                
                l_rate = 0
                o_rate = 0
                g_rate = 0
    
                self.l_rate.append(l_rate)
                self.o_rate.append(o_rate)
                self.g_rate.append(g_rate)
                
    def calculate_sum(self):
        
        self.c_l_rate = sum(self.l_rate)
        self.c_o_rate = sum(self.o_rate)
        self.c_g_rate = sum(self.g_rate)
                

In [35]:
# Cell class

class Cell():
    
    def __init__(self, cell_num, well_dict):
        
        self.cell_num = cell_num
        self.well_dict = well_dict
        self.well_obj_list = []
        
        # Cumulative rates
        self.c_l_rate = 0
        self.c_o_rate = 0
        self.c_g_rate = 0
        
        # Average rates 
        self.l_rate = 0
        self.o_rate = 0
        self.g_rate = 0
        
# Filling well objects list by well names    

    def fill_well_obj_list(self):
        
        for well in well_list:
            if well.well_name in self.well_dict.keys():
                self.well_obj_list.append(well)
        
# Calculate cumulative production

    def calculate_production(self):
        
        for well in self.well_obj_list:
            
            self.c_l_rate += well.c_l_rate
            self.c_o_rate += well.c_o_rate
            self.c_g_rate += well.c_g_rate
            
        self.l_rate = self.c_l_rate / days_in_next_month
        self.o_rate = self.c_o_rate / days_in_next_month
        self.g_rate = self.c_g_rate / days_in_next_month


In [36]:
# Create empty well list

well_list = []

for i in range(current_wop_df.shape[0]):
    
    if type(current_wop_df.iloc[i][1]) == type(1):
        
        well_name_value = current_wop_df.iloc[i][1]
        current_WCT_value = current_wop_df.iloc[i][9]
        current_GOR_value = current_wop_df.iloc[i][10]
        current_Pr_value = current_wop_df.iloc[i][12]
        current_PI_value = current_wop_df.iloc[i][13]
        current_BHP_value = current_wop_df.iloc[i][23]
                
        # Create well instance and assign current WOP
        well = Well(well_name = well_name_value, 
                    current_PI=current_PI_value, 
                    current_WCT=current_WCT_value, 
                    current_GOR=current_GOR_value, 
                    current_Pr=current_Pr_value, 
                    current_BHP=current_BHP_value)
        
        # Assigning planned WOP        
        well.assign_planned_wop()
        
        # Assigning schedule
        well.assign_schedule()
        
        # Assigning rates to schedule
        well.assign_rates()
        
        # Calculate sum
        well.calculate_sum()
        
        # Create list of lists
        well_list_of_lists = []
        well_list_of_lists.append(well.l_rate)
        well_list_of_lists.append(well.o_rate)
        well_list_of_lists.append(well.g_rate)
        
        # Create single well df
        df_well = pd.DataFrame(well_list_of_lists, 
                               columns=[i for i in range(1,days_in_next_month+1)], 
                               index = [str(well.well_name) + ' Ql, m3/d',
                                       str(well.well_name) + ' Qo, t/d',
                                        str(well.well_name) + ' Qg, tm3/d'])
        
        # Concatenating df of the every well
        df_well_rates = pd.concat([df_well_rates, df_well])
        
        # Appending to well list
        well_list.append(well)

In [37]:
# Export to Excel

file_name = "Well_rates_next_month.xlsx"
df_well_rates.to_excel(file_name)

In [38]:
# Create cell dictionaties

# Cell 1
cell_1_name = 1
cell_1_dict = {12:1, 3:1, 13:0.5}

# Cell 2
cell_2_name = 2
cell_2_dict = {13:0.5, 5:1, 14:1, 7:1, 15:1, 9:1, 16:1, 11:1, 78:0.5, 11040:0.3}

# Cell 3
cell_3_name = 3
cell_3_dict = {78:0.5, 111:1, 76:1, 112:1, 113:1, 72:122, 122:1, 123:1, 11040:0.3}

# Cell 4
cell_4_name = 4
cell_4_dict = {11032:0.5, 11040:0.3, 11033:1, 11034:1, 11035:1, 11036:1, 11037:1, 11038:1}

# Cell 5
cell_5_name = 5
cell_5_dict = {11009:1, 11007:1, 11006:1, 11001:1, 11008:1, 91:1, 11005:1, 11023:0.2, 89:1, 11015:0.5}

# Cell 6
cell_6_name = 6
cell_6_dict = {11015:0.5, 11002:1, 11003:1, 11004:1, 87:1, 85:1, 83:1, 81:1, 118:1, 79:1, 74:0.5, 11032:0.3}

# Cell 7
cell_7_name = 7
cell_7_dict = {74:0.5, 77:1, 75:1, 73:1, 71:1, 69:1, 11032:0.3}

# Cell 8
cell_8_name = 8
cell_8_dict = {114:1, 115:1, 805:1, 816:1}

# Cell 9
cell_9_name = 9
cell_9_dict = {11023:0.8, 11024:1, 11025:1, 11026:1, 11027:1, 11028:1, 11029:0.5}

In [43]:
# Collect cells dictionaries

cells_dict = {
     cell_1_name: cell_1_dict,
     cell_2_name: cell_2_dict,
     cell_3_name: cell_3_dict,
     cell_4_name: cell_4_dict,
     cell_5_name: cell_5_dict,
     cell_6_name: cell_6_dict,
     cell_7_name: cell_7_dict,
     cell_8_name: cell_8_dict,
     cell_9_name: cell_9_dict
            }

In [40]:
# Cells list 

cell_list = []

In [41]:
# Create instance of cell classes

for cell in cells_dict.keys():
    
    cell = Cell(cell, cells_dict[cell])
    cell.fill_well_obj_list()
    cell.calculate_production()
    
    cell_list.append(cell)

In [42]:
# Print out results

for cell in cell_list:
    
    print(f"Cumulative rates Cell {cell.cell_num}: \n cQl {cell.c_l_rate} \n cQo {cell.c_o_rate} \n cQg {cell.c_g_rate}")

Cumulative rates Cell 1: 
 cQl 12441.24564158955 
 cQo 4950.638118088444 
 cQg 7014.494662217047
Cumulative rates Cell 2: 
 cQl 12330.163314667736 
 cQo 4774.862012149169 
 cQg 3878.4788189345695
Cumulative rates Cell 3: 
 cQl 7165.950554084049 
 cQo 1601.1891760496899 
 cQg 1041.3681744496728
Cumulative rates Cell 4: 
 cQl 11088.805144464652 
 cQo 4974.633995935027 
 cQg 3382.3097656435602
Cumulative rates Cell 5: 
 cQl 24528.674543610545 
 cQo 14162.944098881557 
 cQg 2653.0006248615505
Cumulative rates Cell 6: 
 cQl 23782.559491377524 
 cQo 12925.01998240039 
 cQg 3932.096604129627
Cumulative rates Cell 7: 
 cQl 28839.81848089468 
 cQo 5493.112432202608 
 cQg 1588.940367228658
Cumulative rates Cell 8: 
 cQl 9842.624162134169 
 cQo 2371.5222926711267 
 cQg 1752.2553749564922
Cumulative rates Cell 9: 
 cQl 7507.209220840398 
 cQo 5159.105420037133 
 cQg 1501.2834201391343
